In [1]:
import extFunc as ext
import numpy as np
from importlib import import_module
import pandas as pd 
import cobra
import optlang
import copy 
import random

In [2]:
#Load Models if not already in RAM

#nameList = ['iJN1462_GLC_UR10_9.xml' , 'iJN1462_GLC_UR6_3.xml', 'iJN1462_GLN_UR5_1.xml', 'iJN1462_GLC_UR7_3.xml' , 'iJN1462_OCT_UR3_4.xml']
nameList = ['iJN1462_GLC_UR6_3.xml' , 'iJN1462_OCT_UR3_4.xml']
#nameList = ['iJN1462_OCT_UR3_4.xml']

if 'modelDict' not in locals():
    modelDict = ext.ImportFunction(nameList)



if 'glc_oct_comp'  not in locals():
    glc_oct_comp = ext.ModelComparison(modelDict=modelDict)

#import extFunc as ext


#a.keys()
#a['secretion'].sort_values('flux', ascending=True).iloc[0:5].reset_index(drop=True)

glc_oct_comp.multiModellSummary(sortKrit='flux')


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


Uptake                                              \
                    metabolite     reaction       flux C-Number     C-Flux   
iJN1462_GLC_UR6_3 0      o2[e]      EX_o2_e  11.985289        0   0.000000   
                  1     nh4[e]     EX_nh4_e   6.552884        0   0.000000   
                  2  glc__D[e]  EX_glc__D_e   6.300000        6  99.994916   
                  3      pi[e]      EX_pi_e   0.566296        0   0.000000   
                  4     so4[e]     EX_so4_e   0.114504        0   0.000000   
iJN1462_OCT_UR3_4 0      o2[e]      EX_o2_e   9.150592        0   0.000000   
                  1    octa[e]    EX_octa_e   3.400000        8  99.996657   
                  2     nh4[e]     EX_nh4_e   3.100000        0   0.000000   
                  3    PHAg[c]    SK_PHAg_c   1.117172        0   0.000000   
                  4      pi[e]      EX_pi_e   0.267900        0   0.000000   

                         Secretion                                      \
                        metabolite         reaction      flux C-Number   
iJN1462_GLC_UR6_3 0        amob[c]        DM_amob_c -0.000137       15   
                  1      doxopa[c]      DM_doxopa_c -0.000137        3   
                  2  tripeptide[c]  DM_tripeptide_c -0.000137        0   
                  3       5drib[c]       DM_5drib_c -0.000412        5   
                  4           h[e]           EX_h_e -6.026947        0   
iJN1462_OCT_UR3_4 0        amob[c]        DM_amob_c -0.000065       15   
                  1      doxopa[c]      DM_doxopa_c -0.000065        3   
                  2  tripeptide[c]  DM_tripeptide_c -0.000065        0   
                  3        meoh[e]        EX_meoh_e -0.000065        1   
                  4       5drib[c]       DM_5drib_c -0.000195        5   

                               
                       C-Flux  
iJN1462_GLC_UR6_3 0  0.015825  
                  1  0.003165  
                  2  0.000000  
                  3  0.015825  
                  4  0.000000  
iJN1462_OCT_UR3_4 0  0.006296  
                  1  0.001259  
                  2  0.000000  
                  3  0.000420  
                  4  0.006296

In [3]:
model = modelDict['iJN1462_OCT_UR3_4']
type(model.boundary)
type(model.exchanges)

#cobra.core.dictlist.DictList(set(model.reactions) - set(model.boundary))
for rxn in model.reactions[:2]:
    print(rxn.metabolites)

{<Metabolite 3hpalmACP[c] at 0x7fa2ea7ba520>: -1.0, <Metabolite h2o[c] at 0x7fa30a20bd60>: 1.0, <Metabolite tpalm2eACP[c] at 0x7fa2ea0df370>: 1.0}
{<Metabolite 13dampp[p] at 0x7fa30a035df0>: -1.0, <Metabolite atp[c] at 0x7fa2ea7c5880>: -1.0, <Metabolite h2o[c] at 0x7fa30a20bd60>: -1.0, <Metabolite 13dampp[c] at 0x7fa309ff3460>: 1.0, <Metabolite adp[c] at 0x7fa2ea7c22b0>: 1.0, <Metabolite h[c] at 0x7fa30a201130>: 1.0, <Metabolite pi[c] at 0x7fa30a1498e0>: 1.0}


In [11]:

a = model.summary()
solution = model.optimize()

a._boundary: List["Reaction"] = [
    #### reaction ### hier veränder damit das Programm die Summary für die internen Flüße macht
    rxn.copy() for rxn in sorted(model.boundary, key=attrgetter("id"))
    #rxn.copy() for rxn in sorted(model.reactions, key=attrgetter("id"))
]
a._boundary_metabolites: List["Metabolite"] = [
    met.copy() for rxn in self._boundary for met in rxn.metabolites

flux = pd.DataFrame(
    data=[
        (rxn.id, met.id, rxn.get_coefficient(met.id), solution[rxn.id])
        for rxn, met in zip(a._boundary, a._boundary_metabolites)
    ],
    columns=["reaction", "metabolite", "factor", "flux"],
    index=[r.id for r in a._boundary],
)

flux.head()


SyntaxError: invalid syntax (2375211847.py, line 12)